<a href="https://colab.research.google.com/github/mb0484/ear_recognition/blob/main/Ear_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="nJmICDv4KQLTtXd7L86q")
project = rf.workspace().project("ear-classification")
dataset = project.version(7).download("folder")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Ear-classification-7 in folder:: 100%|██████████| 2753/2753 [00:02<00:00, 1346.12it/s]


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="nJmICDv4KQLTtXd7L86q")
project = rf.workspace().project("ear-classification")
dataset = project.version(5).download("folder")

     |████████████████████████████████| 145 kB 4.9 MB/s 
     |████████████████████████████████| 178 kB 55.8 MB/s 
     |████████████████████████████████| 1.1 MB 75.5 MB/s 
     |████████████████████████████████| 67 kB 8.3 MB/s 
     |████████████████████████████████| 138 kB 59.4 MB/s 
     |████████████████████████████████| 596 kB 55.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.1-py3-none-any.whl size=20916 sha256=d323d27e758d3d697f2e0058d28fc174bd17c525d5d58fb36bfca6803e9f1848
  Stored in directory: /root/.cache/pip/wheels/57/00/03/2d83f3e90ff41c36eef9c3747c328290c01b06e2619f9ed7b6
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=e594802cac915fdd882a6c176933458390af546fb0ac50b912086a5c2c18434a
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wget
  Attempting uninstall: urllib3
    Found existing inst

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Ear-classification-5 in folder:: 100%|██████████| 2713/2713 [00:01<00:00, 2089.75it/s]


In [3]:
%cd Ear-classification-5

/content/Ear-classification-5


In [4]:
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import os
import torch
import tarfile
import torchvision
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_url

device = ("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_df = pd.DataFrame(columns=["img_name","label"])

train_array = []
train_label_array = []

for dir in os.listdir("train/"):
  cur_class = dir
  for img_name in os.listdir("train/" + dir + "/"):
    train_array.append(img_name)
    train_label_array.append(dir)

train_df["img_name"] = train_array
train_df["label"] = train_label_array

train_df.to_csv (r'train_csv.csv', index = False, header=True)

test_df = pd.DataFrame(columns=["img_name","label"])

test_array = []
test_label_array = []

for dir in os.listdir("test/"):
  cur_class = dir
  for img_name in os.listdir("test/" + dir + "/"):
    test_array.append(img_name)
    test_label_array.append(dir)

test_df["img_name"] = test_array
test_df["label"] = test_label_array

test_df.to_csv (r'test_csv.csv', index = False, header=True)

valid_df = pd.DataFrame(columns=["img_name","label"])

valid_array = []
valid_label_array = []

for dir in os.listdir("valid/"):
  cur_class = dir
  for img_name in os.listdir("valid/" + dir + "/"):
    valid_array.append(img_name)
    valid_label_array.append(dir)

valid_df["img_name"] = valid_array
valid_df["label"] = valid_label_array

valid_df.to_csv (r'valid_csv.csv', index = False, header=True)

In [6]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class CustomEarsDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        image_label = self.annotations.iloc[index, 1]
        new_root_path = self.root_dir + "/" + str(image_label)
        img = Image.open(os.path.join(new_root_path, img_id)).convert("RGB")
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [7]:
transform = transforms.Compose(
        [
            #transforms.Resize((356, 356)),
            #transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [8]:
batch_size = 32
shuffle = True
pin_memory = True
num_workers = 2

In [9]:
train_set = CustomEarsDataset("train","train_csv.csv",transform=transform)
validation_set = CustomEarsDataset("valid","valid_csv.csv",transform=transform)
test_set = CustomEarsDataset("test","test_csv.csv",transform=transform)
train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)
test_loader = DataLoader(dataset=test_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)

In [10]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [11]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, Flatten
from torch.optim import Adam, SGD

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            #BatchNorm2d(64),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(128),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(115200, 1024), #4 * 7 * 7
            ReLU(),
            Dropout(p=0.6, inplace=False),
            Linear(1024, 101)
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

class NetEars(Module):   
    def __init__(self):
        super(NetEars, self).__init__()

        self.cnn_layers = Sequential(
            Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(64),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(256),
            ReLU(inplace=True),
        )

        self.linear_layers = Sequential(
            Linear(2560, 101),
            #ReLU(),
            #Dropout(p=0.6, inplace=False),
            #Linear(1024, 101),
            #Softmax(dim = 1)
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

class ImprovedNet(Module):   
    def __init__(self):
        super(ImprovedNet, self).__init__()

        self.cnn_layers = Sequential(
            Conv2d(3, 32, kernel_size=9, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            BatchNorm2d(32),
            Dropout(0.5),

            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            BatchNorm2d(64),
            Dropout(0.5),

            Conv2d(64, 96, kernel_size=(3,3)),
            ReLU(),
            BatchNorm2d(96),
            Dropout(0.4),
        )

        self.linear_layers = Sequential(
            Flatten(),
            Linear(62976, 101)
        )
 
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [12]:
def rank1(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [13]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def test_step(self, batch, function=rank1):
        images, labels = batch
        out = self(images)                    # Generate predictions
        acc = function(out, labels)           # Calculate accuracy
        return {'test_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def test_epoch_end(self, outputs):
        batch_accs = [x['test_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'test_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [14]:
class MyModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = myNet

    def forward(self, xb):
        return self.network(xb)

In [15]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [16]:
device = get_default_device()
device

device(type='cuda')

In [17]:
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [18]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [19]:
train_dl = DeviceDataLoader(train_loader, device)
val_dl = DeviceDataLoader(validation_loader, device)
test_dl = DeviceDataLoader(test_loader, device)

In [20]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def test(model, test_loader, function=rank1):
    model.eval()
    outputs = [model.test_step(batch, function) for batch in test_loader]
    return model.test_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        model.train() 
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        print("validation")
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [47]:
myNet = Net()
my_model = MyModel()
to_device(my_model, device)

MyModel(
  (network): Net(
    (cnn_layers): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (linear_layers): Sequential(
      (0): Linear(in_features=115200, out_features=1024, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.6, inplace=False)
      (3): Linear(in_features=1024, out_features=101, bias=True)
    )
  )
)

In [35]:
evaluate(my_model, val_dl)

{'val_acc': 0.010416666977107525, 'val_loss': 4.619368076324463}

In [48]:
num_epochs = 45
opt_func = torch.optim.Adam
lr = 0.00001
history = fit(num_epochs, lr, my_model, train_dl, val_dl, opt_func)

validation
Epoch [0], train_loss: 4.5302, val_loss: 4.5305, val_acc: 0.0208
validation
Epoch [1], train_loss: 3.9886, val_loss: 4.3355, val_acc: 0.0312
validation
Epoch [2], train_loss: 3.4642, val_loss: 4.2235, val_acc: 0.1231
validation
Epoch [3], train_loss: 2.8870, val_loss: 4.0282, val_acc: 0.1231
validation
Epoch [4], train_loss: 2.2922, val_loss: 3.9633, val_acc: 0.1250
validation
Epoch [5], train_loss: 1.8494, val_loss: 3.7323, val_acc: 0.1761
validation
Epoch [6], train_loss: 1.4237, val_loss: 3.7475, val_acc: 0.1960
validation
Epoch [7], train_loss: 1.1038, val_loss: 3.5614, val_acc: 0.1970
validation
Epoch [8], train_loss: 0.8296, val_loss: 3.4574, val_acc: 0.3390
validation
Epoch [9], train_loss: 0.6477, val_loss: 3.5781, val_acc: 0.2282
validation
Epoch [10], train_loss: 0.5217, val_loss: 3.2743, val_acc: 0.3589
validation
Epoch [11], train_loss: 0.4091, val_loss: 3.1447, val_acc: 0.3191
validation
Epoch [12], train_loss: 0.3399, val_loss: 3.3329, val_acc: 0.3191
validatio

In [54]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.000001
history = fit(num_epochs, lr, my_model, train_dl, val_dl, opt_func)

validation
Epoch [0], train_loss: 0.0122, val_loss: 2.8751, val_acc: 0.3712
validation
Epoch [1], train_loss: 0.0121, val_loss: 2.8998, val_acc: 0.3807
validation
Epoch [2], train_loss: 0.0115, val_loss: 3.0178, val_acc: 0.3608
validation
Epoch [3], train_loss: 0.0115, val_loss: 2.9712, val_acc: 0.3409
validation
Epoch [4], train_loss: 0.0105, val_loss: 2.8577, val_acc: 0.4205
validation
Epoch [5], train_loss: 0.0103, val_loss: 2.9237, val_acc: 0.4006
validation
Epoch [6], train_loss: 0.0093, val_loss: 3.2312, val_acc: 0.3106
validation
Epoch [7], train_loss: 0.0095, val_loss: 3.1258, val_acc: 0.3513
validation
Epoch [8], train_loss: 0.0113, val_loss: 2.8488, val_acc: 0.4006
validation
Epoch [9], train_loss: 0.0094, val_loss: 3.0040, val_acc: 0.3513


In [26]:
def rank5(outputs, labels):
    _, first_predictions = torch.kthvalue(outputs, k=len(outputs[0]) - 0, dim=1)
    _, second_predictions = torch.kthvalue(outputs, k=len(outputs[0]) - 1, dim=1)
    _, third_predictions = torch.kthvalue(outputs, k=len(outputs[0]) - 2, dim=1)
    _, fourth_predictions = torch.kthvalue(outputs, k=len(outputs[0]) - 3, dim=1)
    _, fifth_predictions = torch.kthvalue(outputs, k=len(outputs[0]) - 4, dim=1)

    sum_right_k_5 = 0;
    sum_right_k_5 += torch.sum(first_predictions == labels).item();
    sum_right_k_5 += torch.sum(second_predictions == labels).item();
    sum_right_k_5 += torch.sum(third_predictions == labels).item();
    sum_right_k_5 += torch.sum(fourth_predictions == labels).item();
    sum_right_k_5 += torch.sum(fifth_predictions == labels).item();

    return torch.tensor(sum_right_k_5 / len(labels))

In [59]:
to_device(my_model, device)
test(my_model, test_dl, function=rank1)

{'test_acc': 0.2906250059604645}

In [60]:
to_device(my_model, device)
test(my_model, test_dl, function=rank5)

{'test_acc': 0.5003471970558167}

In [53]:
torch.save(my_model.state_dict(), 'model_trained_29.pth')

In [57]:
my_model = MyModel()
my_model.load_state_dict(torch.load('model_trained_29.pth'))

<All keys matched successfully>